In [1]:
from scipy.spatial import distance as dst
from collections import defaultdict,Counter,deque
import matplotlib.pyplot as plt
import numpy.random as npr
import igraph as ig
import pickle as pk
import itertools
import random
import heapq
import time
from collections import OrderedDict
import copy
import time
import math

In [2]:
WeightedSectorGraph=pk.load(open("Outputs/WeightedSectorGraph.pkl","rb"))
Centroids = pk.load(open("Outputs/CentroidDict.pkl", "rb"))
#print(G)

In [3]:
from collections import deque
class Chromosome:
    def __init__(self,path,fitness):
        self.path=path
        self.fitness=fitness
def chunks(lst, n):
    B=[]
    B.append(lst[:n])
    for i in range(n, len(lst), n-1):
        tmp=[]
        tmp.append(B[-1][-1])
        tmp+=lst[i:i+n-1]
        B.append(tmp)
    retPairArr=[]
    for i in B:
        retPairArr.append([i[0],i[-1]])
    return retPairArr

In [4]:
#ChangWook Algorithm
from multiprocessing import Process,Pipe
def ChangWookCrossover(G,ChromosomeA, ChromosomeB,startTime,cost):
    setB=set(ChromosomeB.path)
    CommonVertex = [x for x in ChromosomeA.path if x in setB]
    if(len(CommonVertex)<=2):
        return [ChromosomeA, ChromosomeB]
    CommonVertex.pop(0)
    CommonVertex.pop()
    IndexWithCommonVertex=[]
    for vertex in CommonVertex:
        IndexWithCommonVertex.append((ChromosomeA.path.index(vertex),ChromosomeB.path.index(vertex)))
    CrossoverIndexPair=random.choice(IndexWithCommonVertex)
    Path1=ChromosomeA.path[:CrossoverIndexPair[0]+1]+ChromosomeB.path[CrossoverIndexPair[1]+1:]
    Path2=ChromosomeB.path[:CrossoverIndexPair[1]+1]+ChromosomeA.path[CrossoverIndexPair[0]+1:]
    
    return [Chromosome(Path1,PathFitness(G,Path1,startTime,cost)), Chromosome(Path2,PathFitness(G,Path2,startTime,cost))]

    #1 2 3 4 5 6 9 -> 1 2 3 4 6 7 9 
    #  1 2 8 5 6 7 9 -> 1 2 8 4 5 6 9

def ChangWookSelection(Population,SelectionSize):
    size=int(SelectionSize)
    SelectedChromosomes=[]
    NumSelected=0
    PopulationCopy = copy.deepcopy(Population)
    while(NumSelected!=size):
        IndexPair=random.sample(range(len(PopulationCopy)),2)
        PickedIndex=IndexPair[1]
        if(PopulationCopy[IndexPair[0]].fitness>=PopulationCopy[IndexPair[1]].fitness):
            PickedIndex=IndexPair[0]
        SelectedChromosomes.append(copy.deepcopy(PopulationCopy[PickedIndex]))
        NumSelected+=1
        PopulationCopy.pop(PickedIndex)
    return SelectedChromosomes

    # 2 Random Chromosomes chm1>chm2 -> SelectedChm.append(chm1)

def ChangWookMutation(G,Population,start,end,MutationRate,startTime,cost):
    PopulationAvail = sorted(Population, key=lambda x: x.fitness, reverse=True)
    PopulationAvail=PopulationAvail[int(len(Population)/2):]
    if(len(PopulationAvail)==0):
        return
    MutationChance=(1-MutationRate)
    MutationProbability=random.random()
    if MutationProbability>=MutationChance:
        chromosome=random.choice(PopulationAvail)
        StartIndex=random.choice(range(len(chromosome.path)-1))
        StartVertex=chromosome.path[StartIndex]
        chromosome.path=chromosome.path[:StartIndex]
        Newpath=[StartVertex]
        UnionSet=set()
        UnionSet.add(StartVertex)
        while(True):
            viableSet=set(G.neighbors(Newpath[-1]))-UnionSet
            if(len(viableSet)==0):
                Newpath=[StartVertex]
                UnionSet=set()
                UnionSet.add(StartVertex)
                continue
            else:
                ChosenNeighbor=random.choice(tuple(viableSet))
                Newpath.append(ChosenNeighbor)
                UnionSet.add(ChosenNeighbor)
                if(Newpath[-1]==end):
                    chromosome.path+=Newpath
                    chromosome.fitness=PathFitness(G,chromosome.path,startTime,cost)
                    break
    # 1 2 3 4 5 6 7 8 9 OD(1,9) chose 1 2 3 4 5 '6' 7 8 9 -> 1 2 3 4 5 '6 10 11 12 9'
def ChangWookRepair(G,population,startTime,cost):
    for chromosome in population:
        d=defaultdict(int)
        newpath=[]
        for index,vertex in enumerate(chromosome.path):
            d[vertex]=index
        index=0
        while(index<len(chromosome.path)):
            newpath.append(chromosome.path[index])
            index=d[chromosome.path[index]]+1
        chromosome.path=[*newpath]
        chromosome.fitness=PathFitness(G,chromosome.path,startTime,cost)

def getInitPopulation(G,send_end,population_size,start,end,startTime,cost):
    Population=[]
    gotInitPath=False
    while not gotInitPath:
        path=[start]
        pathSet=set(path)
        while not gotInitPath:
            viableSet=set(G.neighbors(path[-1]))-pathSet
            if(len(viableSet)==0):
                path=[start]
                pathSet=set(path)
            else:
                pickedVertex=random.choice(tuple(viableSet))
                path.append(pickedVertex)
                pathSet.add(pickedVertex)
                if(path[-1]==end):
                    Population.append(Chromosome(path,PathFitness(G,path,startTime,cost)))
                    gotInitPath=True
    chunkSize=max(10,int(len(Population[-1].path)/3))
    cks=chunks(path,chunkSize)
    
    while len(Population)!=population_size:
        path=[start]
        for cnkNum in range(len(cks)):
            cnkSt=cks[cnkNum][0]
            cnkEd=cks[cnkNum][1]
            partpath=[cnkSt]
            unionSet=set(partpath)
            while(partpath[-1]!=cnkEd):        
                viableSet=set(G.neighbors(partpath[-1]))-unionSet
                if(len(viableSet)==0):
                    partpath=[cnkSt]
                    unionSet=set(partpath)
                else:
                    nextVer=random.choice(tuple(viableSet))
                    partpath.append(nextVer)
                    unionSet.add(nextVer)
            path.pop()
            path+=partpath   
        newpath=[]
        d=defaultdict(int)
        for index,vertex in enumerate(path):
            d[vertex]=index
        index=0
        while(index<len(path)):
            newpath.append(path[index])
            index=d[path[index]]+1
        Population.append(Chromosome(newpath,PathFitness(G,newpath,startTime,cost)))
    send_end.send(Population)

def InitialPopulation(G,Population,PopulationSize,Start,End,startTime,cost):
    jobs = []
    pipe_list = []
    for i in range(8):
        recv_end, send_end = Pipe(False)
        p = Process(target=getInitPopulation, args=(WeightedSectorGraph,send_end,int(PopulationSize/8),Start,End,startTime,cost))
        jobs.append(p)
        pipe_list.append(recv_end)
        p.start()

    result_list = [x.recv() for x in pipe_list]
    for proc in jobs:
        proc.join()
    for df in result_list:
        Population+=df

def ChangWook(G,start,end,PopulationSize,NumberOfGenerations,MutationRate,SelectionSize,startTime,cost):
    Population=[]
    #Inital Population
    initst=time.time()
    InitialPopulation(G,Population,PopulationSize,start,end,startTime,cost)
    initen=time.time()
#     print("Init Pop",initen-initst)
    Genst=time.time()
    PreDiff=0
    SelDiff=0
    CroDiff=0
    MutDiff=0
    RepDiff=0
    EliDiff=0
    gennum=0
    for Generation in range(NumberOfGenerations):
        gennum+=1
        prest=time.time()
        seen=[]
        PopNoDupli=[]
        Extra=None
        for chromosome in Population:
            if(chromosome.path not in seen):
                seen.append(chromosome.path)
                PopNoDupli.append(chromosome)
            else:
                Extra=chromosome
        if(len(PopNoDupli)%2!=0 and Extra is not None):
            PopNoDupli.append(Extra)
        Population=copy.deepcopy(PopNoDupli)
        PopulationSize=len(Population)
        SelectionSize=PopulationSize/2
        preed=time.time()
        PreDiff+=(preed-prest)
        if(PopulationSize<=2):
            break
        #Selection
        selst=time.time()
        SelectedChm=ChangWookSelection(Population,SelectionSize)
        seled=time.time()
        SelDiff+=(seled-selst)
        
        
        #Crossover
        crost=time.time()
        while(len(SelectedChm)>1):
            cross=random.sample(SelectedChm,2)
            NewChm=ChangWookCrossover(G,cross[0],cross[1],startTime,cost)               
            SelectedChm.remove(cross[0])
            SelectedChm.remove(cross[1])
            Population+=NewChm
        croed=time.time()
        CroDiff+=(croed-crost)
        
        #Mutation
        mutst=time.time()
        ChangWookMutation(G,Population,start,end,MutationRate,startTime,cost)
        muten=time.time()
        MutDiff+=(muten-mutst)
        
        #Repair
        repst=time.time()
        ChangWookRepair(G,Population,startTime,cost)
        repen=time.time()
        RepDiff+=(repen-repst)
        
        #Elimination
        elist=time.time()
        Population = sorted(Population, key=lambda x: x.fitness, reverse=True)[:PopulationSize]
        elien=time.time()
        EliDiff+=(elien-elist)
        
    Genen=time.time()
#     print("Time for all gen",Genen-Genst,"time",startTime)
#     print("Prelim",PreDiff)
#     print("Selection",SelDiff)
#     print("Crossover",CroDiff)
#     print("Mutation",MutDiff)
#     print("Repair",RepDiff)
#     print("Elimination",EliDiff)
#     print("NumberOfGenerations",gennum)
    RetPaths=[]
    RetChm=[]
    for chromosome in Population:
        if(chromosome.path not in RetPaths):
                RetPaths.append(chromosome.path)
                RetChm.append(chromosome)
    RetChm=max(RetChm, key = lambda x: x.fitness)
    return RetChm

In [5]:
def getPaths(G,start,end,startTime,cost,send_end):
    pop_size=4000
    gen_size=50
    Mut_Rate=1
    Selection_Size=pop_size/2
    paths=ChangWook(G,start,end,pop_size,gen_size,Mut_Rate,Selection_Size,startTime,cost)
    send_end.send([paths,startTime])

In [6]:
def GetAngle (p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    dX = x2 - x1
    dY = y2 - y1
    rads = math.atan2 (-dY, dX)
    return math.degrees (rads)
def getAngle(A,B,C):
    pt1=Centroids[A]
    pt2=Centroids[B]
    pt3=Centroids[C]
    a=GetAngle(pt1,pt2)
    b=GetAngle(pt3,pt2)
    if(abs(b-a))>180:
        return 180-(360-abs(b-a))
    else:
        return 180-(abs(b-a))

In [7]:
def PathFitness(G,Path,time,cost):
    global SectorCounts
    cur=Path[0]
    path_length=cost
    for pathIdx in range(1,len(Path)):
        path_length+=G.es[G.get_eid(cur, Path[pathIdx])]["weight"]
        cur=Path[pathIdx] #8,6,9,7 -> Total path len += distance between  any 2 sectors
        
    TrafficFactor=0
    Curtime=time
    for sector in Path:
        TrafficFactor+=SectorCounts[sector][Curtime]
        Curtime+=1
    
    angle=1
    AirportTraffic=SectorCounts[Path[-1]][Curtime-1]+1 
    for i in range(len(Path)-2):
        angle+=getAngle(Path[i],Path[i+1],Path[i+2])
    
    if(TrafficFactor==0):
        return 1/(path_length*angle*AirportTraffic)
    if(TrafficFactor==1):
        return 1/(path_length*(1.5)*angle*AirportTraffic)
    else:
        return 1/(path_length*(TrafficFactor)*angle*AirportTraffic)

In [8]:

SectorCounts = defaultdict(dict,{ k:defaultdict(int) for k in range(1250)})
od=[(6, 662),(6, 662),(6, 662),(6, 662),(6, 662),(6, 662),(6, 662),(6, 662)]
Paths= dict()
initstGA=time.time()
for odPairIdx,odPair in enumerate(od[:8]):
    cost=-10  
    jobs = []
    pipe_list = []
    for i in range(8):
        recv_end, send_end = Pipe(False)
        cost+=10
        p = Process(target=getPaths, args=(WeightedSectorGraph,odPair[0],odPair[1],i,cost,send_end))
        jobs.append(p)
        pipe_list.append(recv_end)
        p.start()

    result_list = [x.recv() for x in pipe_list]
    for proc in jobs:
        proc.join()
    best_chromo=max(result_list,key=lambda x:x[0].fitness)
    Curtime=best_chromo[1]
    for sector in best_chromo[0].path:
        SectorCounts[sector][Curtime]+=1
        Curtime+=1
    Paths[odPairIdx]=best_chromo[0].path
#     print(odPair,Paths[odPairIdx],best_chromo[1])
#     print('\n\n')

# For constant start time
#     pathy=getPaths(WeightedSectorGraph,odPair[0],odPair[1],0,0)
#     Curtime=0
#     for sector in pathy.path:
#         SectorCounts[sector][Curtime]+=1
#         Curtime+=1
#     Paths[odPairIdx]=pathy.path
#     print(odPair,pathy.path)
#     print('\n\n')
initenGA=time.time()
print(f"\n\n\n\n GA TIME \t {initenGA-initstGA}")
A=[]
for i in Paths.keys():
    A.append(Paths[i])
print(A)
pk.dump(A,open(f"GA_Output.pkl","wb"))





 GA TIME 	 294.69750213623047
[[6, 45, 48, 151, 152, 156, 155, 181, 182, 162, 160, 163, 361, 408, 409, 401, 423, 429, 426, 440, 441, 446, 671, 670, 675, 663, 662], [6, 45, 48, 151, 152, 156, 155, 181, 182, 162, 160, 163, 361, 366, 364, 368, 358, 351, 428, 448, 445, 449, 674, 678, 673, 675, 663, 662], [6, 45, 48, 151, 152, 156, 153, 181, 183, 187, 161, 169, 168, 361, 408, 409, 401, 423, 425, 424, 438, 437, 431, 439, 683, 685, 687, 669, 662], [6, 9, 8, 17, 16, 10, 176, 175, 177, 180, 182, 162, 165, 212, 211, 210, 402, 405, 401, 423, 425, 424, 438, 437, 431, 439, 683, 685, 687, 669, 662], [6, 45, 44, 47, 13, 176, 175, 177, 180, 182, 187, 195, 199, 193, 369, 360, 367, 358, 420, 421, 426, 433, 435, 432, 679, 672, 676, 667, 669, 662], [6, 45, 44, 47, 13, 176, 175, 173, 172, 186, 209, 200, 217, 211, 210, 402, 405, 401, 423, 429, 430, 434, 436, 688, 681, 686, 667, 663, 662], [6, 45, 48, 106, 101, 107, 105, 157, 185, 189, 190, 192, 199, 168, 361, 408, 409, 401, 423, 425, 424, 438, 437, 431,

In [9]:
#END OF GA DONT MODIFY CELLS BELOW

In [10]:
KVals = pk.load(open("KDict.pkl", "rb"))
sectorTraffic=dict()
sectorMax=dict()
for sector in range(1250):
    sectorSum=0
    sectorTimeSum=0
    trafficMax=0
    for timeT in SectorCounts[sector]:
        if(SectorCounts[sector][timeT]>0):
            sectorSum+=SectorCounts[sector][timeT]
            sectorTimeSum+=1
            trafficMax=max(trafficMax,SectorCounts[sector][timeT])
    sectorMax[sector]=trafficMax
    if(sectorSum)!=0:
        sectorTraffic[sector]=sectorSum/sectorTimeSum
    else:
        sectorTraffic[sector]=0
sectorTrafficSorted = dict(sorted(sectorTraffic.items(), key=lambda item: item[1],reverse=True))
sectorMaxSorted = dict(sorted(sectorMax.items(), key=lambda item: item[1],reverse=True))
KValsSorted = dict(sorted(KVals.items(), key=lambda item: item[1],reverse=True))
for i in list(sectorTrafficSorted.keys()):
    print(f"{i}\t{sectorTrafficSorted[i]}")

FileNotFoundError: [Errno 2] No such file or directory: 'KDict.pkl'

In [ ]:
sector=662
for timeT in SectorCounts[sector]:
    if(SectorCounts[sector][timeT]>0):
        print(timeT,SectorCounts[sector][timeT])

In [ ]:
#FOR 18 September
KVals = pk.load(open("KDict.pkl", "rb"))
sectorTraffic=dict()
sectorMax=dict()
for sector in range(1250):
    sectorSum=0
    sectorTimeSum=0
    trafficMax=0
    for timeT in SectorCounts[sector]:
        if(SectorCounts[sector][timeT]>0):
            sectorSum+=SectorCounts[sector][timeT]
            sectorTimeSum+=1
            trafficMax=max(trafficMax,SectorCounts[sector][timeT])
    sectorMax[sector]=trafficMax
    if(sectorSum)!=0:
        sectorTraffic[sector]=sectorSum/sectorTimeSum
    else:
        sectorTraffic[sector]=0
sectorTrafficSorted = dict(sorted(sectorTraffic.items(), key=lambda item: item[1],reverse=True))
sectorMaxSorted = dict(sorted(sectorMax.items(), key=lambda item: item[1],reverse=True))
KValsSorted = dict(sorted(KVals.items(), key=lambda item: item[1],reverse=True))
for i in list(sectorTrafficSorted.keys()):
    print(f"{i}\t{sectorTrafficSorted[i]}")

In [ ]:
for i in list(sectorMaxSorted.keys()):
    print(f"{i}\t{sectorMaxSorted[i]}")

In [ ]:
for i in list(KValsSorted.keys()):
    print(f"{i}\t{KVals[i]}")

In [ ]:
GA_Model=list(sectorMaxSorted.values())
K_Orig = list(KValsSorted.values())
fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(20,5))
X=min(min(GA_Model,K_Orig))
fig.suptitle('Traffic Counts Between GA and K')
ax1.hist(GA_Model)
ax1.set_title("Genetic Algorithm")
ax1.set_xlim(X)
ax1.set_ylim(0,1500)
ax2.hist(K_Orig)
ax2.set_title("K")
ax2.set_xlim(X)
ax2.set_ylim(0,1500)

In [ ]:
hotspotSectors=[]
for sector in KValsSorted:
    if(KValsSorted[sector]<=25 and KValsSorted[sector]>=8):
        hotspotSectors.append(sector)
print(len(hotspotSectors))

In [ ]:
hotspotSectorsGA=[]
for sector in sectorMaxSorted:
    if(sectorMaxSorted[sector]<=25 and sectorMaxSorted[sector]>=8):
        hotspotSectorsGA.append(sector)
print(len(hotspotSectorsGA))

In [ ]:
freq=dict()
for count in range(30):
    tr=0
    for sector in sectorMaxSorted:
        if(sectorMaxSorted[sector]==count):
            tr+=1
    freq[count]=tr
for i in freq:
    print(i,freq[i])

In [ ]:
Kfreq=dict()
for count in range(30):
    tr=0
    for sector in KValsSorted:
        if(KValsSorted[sector]==count):
            tr+=1
    Kfreq[count]=tr
for i in Kfreq:
    print(i,Kfreq[i])

In [ ]:
import pandas as pd
data={"SectorCount":[i for i in range(30)],
     "GA Model":[freq[sector] for sector in range(30)],
     "Real World":[Kfreq[sector] for sector in range(30)]}
SectorDataFrame=pd.DataFrame(data)

In [ ]:
SectorDataFrame

In [ ]:
SectorDataFrame.plot.bar(x='SectorCount',figsize=(20,10),ylabel = 'Number of sectors')

In [ ]:
# def Dijkstra(G,start,end):
#     return G.get_shortest_paths(
#         start,
#         to=end,
#         weights=G.es["weight"],
#         output="vpath",
#     )[0]

In [ ]:
a=[1,2,3]
b=[*a]
print(b)